In [1]:
from time import sleep
from selenium import webdriver
import numpy as np
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import copy
import csv
import datetime
import pandas as pd
import json
from ipywidgets import IntProgress
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

### Запускать только с включенным VPN, иначе не отображаются некоторые года

In [2]:
web = webdriver.Chrome("/Users/yasha_ten/Downloads/chromedriver")

In [3]:
url = 'https://www.oddsportal.com/results/#tennis'
web.get(url)

In [4]:
names = []
tournaments = web.find_element_by_css_selector('tbody').find_elements_by_css_selector("tr")
for tourn in tournaments:
    for name in tourn.find_elements_by_css_selector("td"):
        name = name.text
        if name == 'ATP Next Gen Finals - Milan':
            break
        if 'ATP' in name and 'Challenger' not in name and 'Doubles' not in name and\
        'Davis' not in name and 'Olympic' not in name and 'Laver' not in name and 'Next Gen' not in name:
            names.append(name)
names.pop(names.index("ATP ATP Cup"))
names.pop(names.index("ATP Cup Teams - Men"))

In [6]:
## find links for tournaments
links = []
for name in names:
    link = web.find_element_by_link_text(name).get_attribute('href')
    links.append(link)

In [575]:
def fix_time(df):
    time = []
    date_fx = []
    months = {"Jan": 1, "Feb": 2, "Mar": 3,
              "Apr": 4, "May": 5, "Jun": 6,
              "Jul": 7, "Aug": 8, "Sep": 9,
              "Oct": 10, "Nov": 11, "Dec": 12
             }
    for date in df["date"]:
        mas = date.split()
        date_fx.append(datetime.date(int(mas[2][:-1]), months[mas[1]], int(mas[0])))
        podmas = mas[-1].split(":")
        time.append(datetime.time(int(podmas[0]), int(podmas[1]), 0))
    df["time"] = time
    df["date"] = date_fx
    
def fix_letters(df_):
    progress = IntProgress(min=0, max=len(df_), value=0)
    display(progress)
    bug_surnames = {'Hüsler M.': 'Huesler M.', 'Mcdonald M.': 'McDonald M.',
                    'del Potro J.': 'Del Potro J.', 'López-Pérez E.': 'Lopez-Perez E.',
                   'Di Wu.': 'Wu D.', 'Yen-Hsun Lu.': 'Lu Y.H.', 'Mcgee J.': 'McGee J.',
                   'Zhe Li.': 'Li Z.', 'Wolf J.J.': 'Wolf J.', 'Bautista Agut R.': 'Bautista-Agut R.',
                   'Ramos-Viñolas A.': 'Ramos A.', 'Carreño Busta P.': 'Carreno-Busta P.', 'Struff J.': 'Struff J-L.',
                   'Tsonga J.': 'Tsonga J-W.', 'de Minaur A.': 'De Minaur A.', 'Andújar P.': 'Andujar-Alba P.',
                    'Dutra Silva R.': 'Dutra Da Silva D.', 'Smith J.': 'Smith J. P.', 'Estrella Burgos V.': 'Estrella V.',
                    'Popyrin A.': 'Popyrin Al.', 'Londero J.': 'Londero J. I.', 'Harris L.': 'Harris G.',
                    'de Bakker T.': 'De Bakker T.', 'Kwon S.': 'Kwon Soonwoo', 'Galán D.': 'Galan Riveros D. E.',
                    'Lee D.': 'Lee D. H.', 'Muñoz de la Nava D.': 'Munoz-De La Nava D.', 'Ojeda Lara R.': 'Ojeda L. R.',
                    'Kuznetsov A.': 'Kuznetsov Al.', 'Lindell C.': 'Lindell Ch.', 'Vilella Martínez M.': 'Vilella M. M.',
                    'Kwiatkowski T.': 'Kwiatkowski T. S.', 'Moroni G.': 'Moroni M.',
                    'Huta Galung J.': 'Huta-Galung J.', 'Mukund S.': 'Mukund S. K.', 'Ramirez Hidalgo R.': 'Ramirez-H.R.',
                    'Tyurnev E.': 'Tiurnev E.', 'Yecong He.': 'He Y.', 'Statham R.': 'Statham J. R.',
                   'Rigele Te.': 'Te R.', 'Prashanth N.': 'Prashanth V.', 'Yibing W.': 'Wu Y.',
                    'Varillas J.': 'Varillas J. P.', 'Ficovich J.': 'Ficovich J. P.', 'Lipovšek Puches T.': 'Lipovsek P. T.',
                    'Tatlot J.': 'Tatlot J. S.', 'Sorgi J.': 'Sorgi J. P.', 'Zayed M.': 'Zayed M. S.'}
    letters = {'á': 'a', 'ã': 'a', 'ç': 'c',
           'é': 'e', 'í': 'i', 'ñ': 'n',
           'ó': 'o', 'ö': 'o', 'ú': 'u',
          'ü': 'u', 'ý': 'y', 'ć': 'c',
          'č': 'c', 'ě': 'e', 'ı': 'i',
          'ł': 'l', 'ř': 'r', 'š': 's',
          'ž': 'z'}
    for i in range(len(df_)):
        p1 = df_['player1'][i]
        p2 = df_['player2'][i]
        p2.replace('\n', '') ## лишние \n при парсинге oddsportal
        
        if p1 in bug_surnames:
            p1 = bug_surnames[p1]
            
        if p2 in bug_surnames:
            p2 = bug_surnames[p2]

        for c in range(len(p1)):
            if p1[c].lower() in letters:
                if p1[c].islower():
                    p1 = p1.replace(p1[c], letters[p1[c]])
                else:
                    p1 = p1.replace(p1[c], letters[p1[c].lower()].upper())

        for c in range(len(p2)):
            if p2[c].lower() in letters:
                if p2[c].islower():
                    p2 = p2.replace(p2[c], letters[p2[c]])                    
                else:
                    p2 = p2.replace(p2[c], letters[p2[c].lower()].upper())

        df_['player1'][i] = p1
        df_['player2'][i] = p2
        
        if i % 10 == 0:
            progress.value = i

In [6]:
def write_arr(arr, filename):
    with open(filename, 'w') as f:
        json.dump(arr, f)

In [7]:
# читаем из файла
def read_file(filename):
    with open(filename, 'r') as f:
        arr = json.load(f)
    return arr

In [8]:
def wait_until_cond_css(css):
        flag = 0
        while True:
            try:
                web.find_element_by_css_selector(css)
                break
            except Exception:
                if (flag < 20):
                    sleep(0.5)
                    flag += 1
                else:
                    raise NoSuchElementException
                    
def wait_until_cond_link(link):
        flag = 0
        while True:
            try:
                web.find_element_by_link_text(link)
                break
            except Exception:
                if (flag < 20):
                    sleep(0.5)
                    flag += 1
                else:
                    raise NoSuchElementException

In [589]:
write_arr(names, "tourn_names.txt")
write_arr(links, "tourn_links.txt")

In [17]:
names = read_file('tourn_names.txt')
links = read_file('tourn_links.txt')

In [20]:
data = []

In [11]:
def parse_matches(matches, urls_):
    i = 0
    for match in matches:
        names_ = match.split(' - ')
        p1 = names_[0]
        p2 = names_[1]
        web.get(urls_[i])
        sleep(0.25)
#             wait_until_cond_link(match)
#             web.find_element_by_link_text(match).click()
# #             act = ActionChains(web)
# #             act.click(el_).perform()
        
    
        if web.find_elements_by_css_selector('center') != []:
            web.refresh()
            sleep(3)
            
        wait_until_cond_css("p.date.datet")
        date = web.find_element_by_css_selector("p.date.datet").text.split(',')[1].split()
        date = datetime.date(int(date[2]), months[date[1]], int(date[0])) ## match date

        coefs = web.find_elements_by_css_selector("tr.lo.odd") ## find coefs odd_
        elem = {'player1': p1, 'player2': p2, 'date': date}
        for bm in coefs:
            bm = bm.text.split()
            name = bm[0] ## аккуратно в следующий раз, переделать, если имя состоит из нескольких слов, то идет съезд...
            k1 = bm[1]
            k2 = bm[2]
            ## add koefs to match
            elem['k1_' + name] = k1
            elem['k2_' + name] = k2

        coefs = web.find_elements_by_css_selector("tr.lo.even") ## find coefs even_
        for bm in coefs:
            bm = bm.text.split()
            name = bm[0] ## аккуратно в следующий раз, переделать, если имя состоит из нескольких слов, то идет съезд...
            k1 = bm[1]
            k2 = bm[2]
            ## add koefs to match
            elem['k1_' + name] = k1
            elem['k2_' + name] = k2
        try:
            a = web.find_elements_by_css_selector('td.center.disabled')[0].text.split()[0]
            b = web.find_elements_by_css_selector('td.center.disabled')[1].text.split()[0]
            user1 = round(100 / int(a.replace('%', '')), 2)
            user2 = round(100 / int(b.replace('%', '')), 2)
            elem['k1_user'] = user1
            elem['k2_user'] = user2
        except Exception:
            pass

        data.append(elem) ## add match to array
        i += 1

months = {"Jan": 1, "Feb": 2, "Mar": 3,
              "Apr": 4, "May": 5, "Jun": 6,
              "Jul": 7, "Aug": 8, "Sep": 9,
              "Oct": 10, "Nov": 11, "Dec": 12
            }

for link in links:
    web.get(link) ## go to tournament page
    sleep(3)
    if web.find_element_by_css_selector('div#breadcrumb').text != 'The page you requested is not available.':
        years = {}
        wait_until_cond_css('div.main-menu2.main-menu-gray')

        ## find all needed years
        years = {}
        for x in web.find_element_by_css_selector('div.main-menu2.main-menu-gray').find_elements_by_css_selector('li'):
            years[int(x.find_element_by_css_selector('a').text)] = x.find_element_by_css_selector('a').get_attribute('href')

        for year in range(2015, 2021):
            matches = []
            if year in years:
    #             print("Go to ", year)
    #             print(years[year])
                web.get(years[year]) ## go to needed year
                sleep(1)
            else:
                continue

            sleep(1.5)
            pagination = web.find_elements_by_css_selector('div#pagination')

            if pagination != []: ## condition to check whether pages
                page_links = set()
                for x in pagination[0].find_elements_by_css_selector('a'):
                    page_links.add(x.get_attribute('href'))

                for link_ in page_links:
                    matches.clear()
                    urls_.clear()
                    web.get(link_) ## change page
                    wait_until_cond_css('td.name.table-participant')
                    sleep(2)
                    for match in web.find_elements_by_css_selector('td.name.table-participant'): ## add matches on page
                        matches.append(match.text)
                    ## links of this mathces
                    urls_ = [x.find_element_by_css_selector('a').get_attribute('href') for x in web.find_elements_by_css_selector('td.name.table-participant')]
                    ## parse every match by link
                    parse_matches(matches, urls_)

            else:
                for match in web.find_elements_by_css_selector('td.name.table-participant'):
                    matches.append(match.text)

                urls_ = [x.find_element_by_css_selector('a').get_attribute('href') for x in web.find_elements_by_css_selector('td.name.table-participant')]
                parse_matches(matches, urls_)
                
                
def combine(df, df_):
    progress = IntProgress(min=0, max=len(df_), value=0)
    display(progress)

    for i in range(len(df_)):

        player1 = df_['player1'][i]
        player2 = df_['player2'][i]
        date = np.datetime64(df_['date'][i])
        match = df[(df['player1'] == player1) & (df['player2'] == player2) & ((df['date'] == date) | (df['date'] == date + 1) | (df['date'] == date - 1))]

        if len(match) == 1:
            for k in list(df.keys())[3:]: ## k1_1Xbet, ....
                df_.loc[i, k] = float(match[k])

        match = df[(df['player1'] == player2) & (df['player2'] == player1) & ((df['date'] == date) | (df['date'] == date + 1) | (df['date'] == date - 1))]

        if len(match) == 1:
            for k in list(df.keys())[3:]: ## k1_1Xbet, ....
                if 'k1' in k:
                    df_.loc[i, k.replace('k1', 'k2')] = float(match[k])
                else:
                    df_.loc[i, k.replace('k2', 'k1')] = float(match[k])

        if (i % 10 == 0):
            progress.value = i
            
def drop_n_fill_trash(df):
    for k in df.keys():
        if 'k2' in k:
            for i in range(len(df)):
                if '%' in str(df[k][i]):
                    if float(df[k.replace('k2', 'k1')][i]) < 11:
                        df[k][i] = 1.01
                    else:
                        df[k][i] = np.nan
    
    set_ = set()
    for k in df.keys():
        if 'k2' in k:
            for i in range(len(df)):
                if '-' in str(df[k][i]):
                    set_.add(i)
    return set_

In [188]:
df = pd.DataFrame(data).set_index(['date', 'player1', 'player2']).sort_index()
df.to_pickle('all_odds.pkl')

In [807]:
df_ = pd.read_pickle('matches.pkl')
fix_time(df_)
df_ = df_[df_['date'] > np.datetime64('2015-01-01')].set_index('player1').reset_index()
fix_letters(df_)


df = pd.read_pickle('all_odds.pkl')
df = df.reset_index()
df.drop(['k1_Asianodds', 'k2_Asianodds', 'k1_GGBET', 'k2_GGBET', 'k1_William', 'k2_William'], axis=1, inplace=True)

set_ = drop_n_fill_trash(df)
## del trash
for ind in set_:
    df = df.drop(ind)
df = df.set_index('player1').reset_index()
## combine coefs and data
combine(df)

In [9]:
# df_.to_pickle('all_coefs_combine.pkl')
pd.read_pickle('all_coefs_combine.pkl')

,player1,match_url,player2,status,Surface,date,k1,k2,player1_win,round,...,k1_Pinnacle,k2_Pinnacle,k1_user,k2_user,k1_BetWorld,k2_BetWorld,k1_Bethard,k2_Bethard,k1_Coolbet,k2_Coolbet
0,Ilhan M.,https://www.sofascore.com/kovalik-ilhan/pHgswPx,Kovalik J.,Ended,Hardcourt outdoor,2015-01-03,NaN,NaN,1,Qualification,...,1.49,2.74,1.49,3.03,NaN,NaN,NaN,NaN,NaN,NaN
1,Kamke T.,https://www.sofascore.com/mott-kamke/iugsMMw,Mott B.,Ended,Hardcourt outdoor,2015-01-03,NaN,NaN,1,Qualification,...,1.07,9.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kuznetsov Al.,https://www.sofascore.com/kuznetsov-kubot-l/iV...,Kubot L.,Ended,Hardcourt outdoor,2015-01-03,NaN,NaN,0,Qualification,...,2.20,1.71,3.45,1.41,NaN,NaN,NaN,NaN,NaN,NaN
3,Uchiyama Y.,https://www.sofascore.com/pouille-uchiyama/PCt...,Pouille L.,Ended,Hardcourt outdoor,2015-01-03,NaN,NaN,1,Qualification,...,3.78,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Podlipnik-Castillo H.,https://www.sofascore.com/reid-podlipnik-casti...,Reid M.,Ended,Hardcourt outdoor,2015-01-03,NaN,NaN,0,Qualification,...,3.08,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22089,Djokovic N.,https://www.sofascore.com/zverev-djokovic/HXfsnTx,Zverev A.,Ended,Hardcourt indoor,2020-11-20,1.33,3.40,1,A,...,1.32,3.78,1.19,6.25,NaN,NaN,1.33,3.70,1.32,3.78
22090,Medvedev D.,https://www.sofascore.com/schwartzman-medvedev...,Schwartzman D.,Ended,Hardcourt indoor,2020-11-20,1.28,3.75,1,A,...,1.30,3.96,5.56,1.22,NaN,NaN,1.30,3.95,1.31,3.87
22091,Thiem D.,https://www.sofascore.com/thiem-djokovic/HXfsYzr,Djokovic N.,Ended,Hardcourt indoor,2020-11-21,2.37,1.57,1,Semifinals,...,2.70,1.54,1.28,4.55,NaN,NaN,2.65,1.56,2.31,1.62
22092,Medvedev D.,https://www.sofascore.com/medvedev-nadal/LOfsevpb,Nadal R.,Ended,Hardcourt indoor,2020-11-21,1.72,2.10,1,Semifinals,...,1.88,2.04,1.82,2.22,NaN,NaN,1.87,2.10,1.78,2.05


In [826]:
## bm : missed missings
for k in df.keys():
    if 'k1' in k:
        print(k, ': ', len(df) - len(df.dropna(subset=[k])))

k1_1xBet :  150
k1_bet-at-home :  304
k1_Betsafe :  483
k1_bwin :  1646
k1_Marathonbet :  408
k1_Unibet :  1901
k1_888sport :  1470
k1_bet365 :  86
k1_Betway :  561
k1_bwin.fr :  10692
k1_Pinnacle :  142
k1_user :  2769
k1_BetWorld :  6617
k1_Bethard :  9465
k1_Coolbet :  10049
